In [1]:
import datetime as dt

import altair as alt
import altair_morberg.core as morberg
import numpy as np
import pandas as pd
from covid import dataimport

base_url = "https://raw.githubusercontent.com/morberg/covid-notebook/master/data/"
combined_url = base_url + "combined.json"
scb_url = base_url + "scb.json"

alt.themes.register("morberg_theme", morberg.theme)
alt.themes.enable("morberg_theme");

In [2]:
data_scb = dataimport.get_scb_county_data()
latest_data_point = data_scb.date.max()
cutoff = latest_data_point.timetuple().tm_yday - 10
published = f'Data published {latest_data_point.date().isoformat()} by SCB'

# Excess mortality in Sweden 2020

Charts that show excess mortality in 2020 compared to 2018-2019. Based on data [published weekly by Statistiska Centralbyrån](https://scb.se/hitta-statistik/statistik-efter-amne/befolkning/befolkningens-sammansattning/befolkningsstatistik/pong/tabell-och-diagram/preliminar-statistik-over-doda/). Due to reporting lag the data for the last two weeks will change. These data points are still included but shown with less prominence.

## Total weekly deaths 

Numbers for Covid-19 deaths from FHM marked in orange. This gives an idea on how accurate the reporting on Covid-19 deaths is.

In [5]:
base = alt.Chart('data/combined.json', height=400, width=750, title={
        "text": 'Weekly Deaths in Sweden 2018-2020',
        "subtitle": published}).encode(
    x=alt.X('monthdate(date):T', axis=alt.Axis(format="%b"), title='')
)

history = base.mark_line(size=3, opacity=1).transform_filter(
    'year(datum.date) < 2020'
).transform_aggregate(
    sum_deaths='sum(deaths):Q',
    groupby=['date']
).encode(
    y='mean(sum_deaths):Q',
    color=alt.Color('year(date):N', title='', scale=alt.Scale(range=['#b9b9b9', '#898989']))
)

covid_base = base.encode(
    y=alt.Y('deaths:Q', title=''),
    color=alt.Color('death_cause:N', title='2020', sort='descending')
).transform_filter(
    'year(datum.date) == 2020'
)

covid = covid_base.mark_bar(size=14).transform_filter(
    f'dayofyear(datum.date) <= {cutoff}'
)

covid_uncertain = covid_base.mark_bar(size=14,opacity=0.3).encode(
    color=alt.Color('death_cause:N', title='2020', sort='descending', legend=None)
)

(covid + covid_uncertain + history).resolve_scale(color='independent')

alt.Chart(...)

## Cumulative deaths 2018-2020 by County

An illustration of total deaths in 2020 per county compared to 2018 and 2019. Last two weeks of data in 2020 are shown as points since they will change.

In [8]:
base = alt.Chart(scb_url, height=220, width=220).mark_line().encode(
    x=alt.X('monthdate(date):T', title='', axis=alt.Axis(format="%b")),
    y=alt.Y('cumulative_deaths:Q', title=''),
    color=alt.Color('year(date):N', title='', scale=alt.Scale(range=['#d9d9d9', '#c9c9c9','#b9b9b9','#a9a9a9','#999999', '#e6550d']), legend=alt.Legend(orient='top')),
    tooltip=['date:T', alt.Tooltip('cumulative_deaths:Q', title='Deaths')],
)

history = base.transform_filter(
    'year(datum.date) < 2020',
)

current = base.transform_filter(
    f'dayofyear(datum.date) <= {cutoff} && year(datum.date == 2020)'
)

current_uncertain = base.mark_circle(opacity=0.8, size=10).transform_filter(
    f'year(datum.date) == 2020 && dayofyear(datum.date) >= {cutoff}'
)

alt.layer(history, current, current_uncertain).facet(
    alt.Facet('county:N', title=""),
    columns=3,
).resolve_scale(y='independent').properties(title='Total Deaths in Sweden 2018-2020 by County')

alt.FacetChart(...)

## Weekly deaths with filter on county level

Click on a county marker in the legend to the right to include/exclude it in the total numbers.

In [9]:
base = alt.Chart(scb_url, height=400, width=700, title={
        "text": 'Weekly Deaths in Sweden 2018-2020',
        "subtitle": published}).encode(
    x=alt.X('monthdate(date):T', axis=alt.Axis(format="%b"), title=None),
    y=alt.Y('sum(deaths):Q', title='Weekly Deaths'),
)

history = base.mark_line(opacity=0.4, size=2).encode(color=alt.Color('year(date):N', title='Year')).transform_filter(
    "year(datum.date) < 2020"
)

current_base = base.encode().mark_line(size=5, opacity=1).encode(
    y='sum(deaths):Q',
    color=alt.Color('year(date):N', title=None, scale=alt.Scale(range=['#e45757']))
).transform_filter(
    "year(datum.date) == 2020"
)

current = current_base.transform_filter(
    f'dayofyear(datum.date) <= {cutoff}'
)

current_uncertain = current_base.mark_line(size=5, opacity=0.2).encode(
    color=alt.Color('death_cause:N', scale=alt.Scale(range=['#e45757']), legend=None)
)

selection = alt.selection_multi(toggle='true', encodings=['y'])

legend = alt.Chart(scb_url, height=320, width=60).mark_square(size=150).encode(
    y=alt.Y('county:N', axis=alt.Axis(orient='right'), title=None),
    x=alt.X('sum(deaths):Q', title='Total Deaths 2020'),
    color=alt.condition(selection, alt.value('#e45757'), alt.value('lightgrey')),
    tooltip=[alt.Tooltip('sum(deaths):Q', title='Total Deaths 2020'), 'county:N'],
).add_selection(
    selection
).interactive()

(history + current + current_uncertain).transform_filter(selection).resolve_scale(color='independent') | legend

alt.HConcatChart(...)